# 情感分析作业

说明：本次任务是有关自然语言处理领域中的情感识别问题（也叫观点分析问题，也就是将数据分类成正向、负向），包含六万多条购物评价，分别来自书籍、平板、手机、水果、洗发水、热水器、蒙牛、衣服、计算机、酒店，共十个类别。

## 一、数据读入及预处理

In [1]:
import pandas as pd
import jieba
import re
import os

from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

from torch.utils.tensorboard import SummaryWriter 
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pad_packed_sequence

e:\python\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### （1）数据读入

In [2]:
path="online_shopping_10_cats.csv"
df=pd.read_csv(path)
df.head()

,cat,label,review
0,书籍,1,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...
1,书籍,1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2,书籍,1,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3,书籍,1,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4,书籍,1,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...


### （2）数据筛选及处理

In [3]:
df=df[["review","label"]]

df.head()

,review,label
0,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...,1
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...,1
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...,1
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...,1
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...,1


### （3）去重

In [4]:
print(df.shape)
df.drop_duplicates()

(62774, 2)


,review,label
0,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...,1
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...,1
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...,1
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...,1
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...,1
...,...,...
62769,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...,0
62770,房间很小，整体设施老化，和四星的差距很大。毛巾太破旧了。早餐很简陋。房间隔音很差，隔两间房间...,0
62771,我感觉不行。。。性价比很差。不知道是银川都这样还是怎么的！,0
62772,房间时间长，进去有点异味！服务员是不是不够用啊！我在一楼找了半个小时以上才找到自己房间，想找...,0


### （4）数据清洗

In [5]:
info=re.compile("[0-9a-zA-Z]|作者|当当网|京东|洗发水|蒙牛|衣服|酒店|房间")
df["review"]=df["review"].apply(lambda x:info.sub("",str(x)))
df["review"].head()

0    ﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...
1    真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到真理...
2    长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产率？...
3    在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延续，...
4    在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵的有...
Name: review, dtype: object

### （5）分词

In [6]:
df["words"]=df["review"].apply(jieba.lcut)
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.606 seconds.
Prefix dict has been built successfully.


,review,label,words
0,﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...,1,"[﻿, 做, 父母, 一定, 要, 有, 刘墉, 这样, 的, 心态, ，, 不断, 地, ..."
1,真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到真理...,1,"[真有, 英国人, 严谨, 的, 风格, ，, 提出, 观点, 、, 进行, 论述, 论证,..."
2,长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产率？...,1,"[长篇大论, 借用, 详细, 报告, 数据处理, 工作, 和, 计算结果, 支持, 其新, ..."
3,在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延续，...,1,"[在, 战, 几时, 之前, 用, 了, ＂, 拥抱, ＂, 令人, 叫绝, ．, 日本, ..."
4,在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵的有...,1,"[在, 少年, 时即, 喜, 阅读, ，, 能, 看出, 他, 精读, 了, 无数, 经典,..."


### （6）建立词表

In [7]:
words = []
for sentence in df["words"].values:
    for word in sentence:
        words.append(word)

In [8]:
len(words)

2277845

In [9]:
words = list(set(words))

In [10]:
words = sorted(words)

In [11]:
#词表中的单词数
len(words)

64104

In [12]:
word2idx = {w:i+1 for i,w in enumerate(words)}
idx2word = {i+1:w for i,w in enumerate(words)}
word2idx['<unk>'] = 0
idx2word[0] = '<unk>'

### （7）将中文词数字化表示

In [13]:
data = []
label = []
        
for sentence in df['words']:
    words_to_idx = []
    for word in sentence:
        index = word2idx[word]
        words_to_idx.append(index)
        
    data.append(torch.tensor(words_to_idx))
label = torch.from_numpy(df['label'].values)

In [14]:
len(data)

62774

In [15]:
len(label)

62774

### （8）划分训练集和验证集

In [16]:
x_train,x_val,y_train,y_val=train_test_split(data,label,test_size=0.2)

### （9）设置DataSet和DataLoader
提供现成的数据变长处理的方法，可以直接在DataLoader的参数中设置collate_fn=mycollate_fn来使用这个方法

In [17]:
def mycollate_fn(data):
    
    data.sort(key=lambda x: len(x[0]), reverse=True)  
    data_length = [len(sq[0]) for sq in data]
    input_data = []
    label_data = []
    for i in data:
        input_data.append(i[0])
        label_data.append(i[1])
    input_data = pad_sequence(input_data, batch_first=True, padding_value=0)
    label_data = torch.tensor(label_data)
    return input_data, label_data, data_length

In [18]:
class mDataSet(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __getitem__(self, idx: int):
        return self.data[idx], self.label[idx]

    def __len__(self):
        return len(self.data)

In [19]:
train_dataset=mDataSet(x_train, y_train)

In [20]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,num_workers=0,collate_fn=mycollate_fn)

In [21]:
val_dataset = mDataSet(x_val, y_val)

In [22]:
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True,num_workers=0, collate_fn=mycollate_fn)

## 二、建立模型
### （1）定义模型

In [23]:
class Model(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_dim, num_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(num_embeddings,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim,num_layers)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(self.hidden_dim,256)
        self.fc2 = nn.Linear(256,32)
        self.fc3 = nn.Linear(32,2)

    def forward(self, input, batch_seq_len):
        embeds = self.embeddings(input) 
        embeds = pack_padded_sequence(embeds,batch_seq_len, batch_first=True)
        batch_size, seq_len = input.size()
        
        h_0 = input.data.new(3, batch_size, self.hidden_dim).fill_(0).float()
        c_0 = input.data.new(3, batch_size, self.hidden_dim).fill_(0).float()
        
        output, hidden = self.lstm(embeds, (h_0, c_0))
        output,_ = pad_packed_sequence(output,batch_first=True)

        output = self.dropout(torch.tanh(self.fc1(output)))
        output = torch.tanh(self.fc2(output))
        output = self.fc3(output)
        last_outputs = self.get_last_output(output, batch_seq_len)
        return last_outputs,hidden
    
    def get_last_output(self,output,batch_seq_len):
        last_outputs = torch.zeros((output.shape[0],output.shape[2]))
        for i in range(len(batch_seq_len)):
            last_outputs[i] =  output[i][batch_seq_len[i]-1]#index 是长度 -1
        last_outputs = last_outputs.to(output.device)
        return last_outputs

### （2）初始化模型

In [24]:
# 实例化模型

# 定义优化器

# 学习率调整（可选）

# 定义损失函数

In [25]:
#可供参考的模型结构
model

Model(
  (embeddings): Embedding(64105, 50)
  (lstm): LSTM(50, 100, num_layers=3)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)

### （3）准确率指标

In [27]:
class AvgrageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.cnt = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.cnt += n
        self.avg = self.sum / self.cnt

In [28]:
def accuracy(output, label, topk=(1,)):
    maxk = max(topk) 
    batch_size = label.size(0)

    # 获取前K的索引
    _, pred = output.topk(maxk, 1, True, True) #使用topk来获得前k个的索引
    pred = pred.t() # 进行转置
    # eq按照对应元素进行比较 view(1,-1) 自动转换到行为1,的形状， expand_as(pred) 扩展到pred的shape
    # expand_as 执行按行复制来扩展，要保证列相等
    correct = pred.eq(label.view(1, -1).expand_as(pred)) # 与正确标签序列形成的矩阵相比，生成True/False矩阵
#     print(correct)

    rtn = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0) # 前k行的数据 然后平整到1维度，来计算true的总个数
        rtn.append(correct_k.mul_(100.0 / batch_size)) # mul_() ternsor 的乘法  正确的数目/总的数目 乘以100 变成百分比
    return rtn

In [ ]:
class ConfuseMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        # 标签的分类：0 pos 1 neg 
        self.confuse_mat = torch.zeros(2,2)
        self.tp = self.confuse_mat[0,0]
        self.fp = self.confuse_mat[0,1]
        self.tn = self.confuse_mat[1,1]
        self.fn = self.confuse_mat[1,0]
        self.acc = 0
        self.pre = 0
        self.rec = 0
        self.F1 = 0
    def update(self, output, label):
        pred = output.argmax(dim = 1)
        for l, p in zip(label.view(-1),pred.view(-1)):
            self.confuse_mat[p.long(), l.long()] += 1 # 对应的格子加1
        self.tp = self.confuse_mat[0,0]
        self.fp = self.confuse_mat[0,1]
        self.tn = self.confuse_mat[1,1]
        self.fn = self.confuse_mat[1,0]
        self.acc = (self.tp+self.tn) / self.confuse_mat.sum()
        self.pre = self.tp / (self.tp + self.fp)
        self.rec = self.tp / (self.tp + self.fn)
        self.F1 = 2 * self.pre*self.rec / (self.pre + self.rec)

### （4）训练

In [29]:
def train(epoch,epochs, train_loader, device, model, criterion, optimizer,scheduler,tensorboard_path):
    model.train()
    top1 = AvgrageMeter()
    model = model.to(device)
    train_loss = 0.0
    for i, data in enumerate(train_loader, 0):  # 0是下标起始位置默认为0
        inputs, labels, batch_seq_len = data[0].to(device), data[1].to(device), data[2]
        if batch_seq_len[-1] <= 0:
            continue
        # 初始为0，清除上个batch的梯度信息
        optimizer.zero_grad()
        outputs,hidden = model(inputs,batch_seq_len)

        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        _,pred = outputs.topk(1)
        prec1, prec2= accuracy(outputs, labels, topk=(1,2))
        n = inputs.size(0)
        top1.update(prec1.item(), n)
        train_loss += loss.item()
        postfix = {'train_loss': '%.6f' % (train_loss / (i + 1)), 'train_acc': '%.6f' % top1.avg}
        train_loader.set_postfix(log=postfix)

        # ternsorboard 曲线绘制
        if os.path.exists(tensorboard_path) == False: 
            os.mkdir(tensorboard_path)    
        writer = SummaryWriter(tensorboard_path)
        writer.add_scalar('Train/Loss', loss.item(), epoch)
        writer.add_scalar('Train/Accuracy', top1.avg, epoch)
        writer.flush()
    scheduler.step()

In [30]:
epochs=1
for epoch in range(epochs):
    train_loader = tqdm(train_loader)
    train_loader.set_description('[%s%04d/%04d %s%f]' % ('Epoch:', epoch + 1, epochs, 'lr:', scheduler.get_lr()[0]))
    train(epoch, epochs, train_loader, device, model, criterion, optimizer,scheduler, "visual")

  0%|          | 0/3139 [00:00<?, ?it/s]

e:\python\lib\site-packages\torch\optim\lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


In [34]:
def validate(epoch,validate_loader, device, model, criterion, tensorboard_path):
    val_acc = 0.0
    model = model.to(device)
    model.eval()
    with torch.no_grad():  # 进行评测的时候网络不更新梯度
        val_top1 = AvgrageMeter()
        validate_loader = tqdm(validate_loader)
        validate_loss = 0.0
        for i, data in enumerate(validate_loader, 0):  # 0是下标起始位置默认为0
            inputs, labels, batch_seq_len = data[0].to(device), data[1].to(device), data[2]
            if batch_seq_len[-1] <= 0:
                continue
            outputs,_ = model(inputs, batch_seq_len)
            loss = criterion(outputs, labels)

            prec1, prec2 = accuracy(outputs, labels, topk=(1, 2))
            n = inputs.size(0)
            val_top1.update(prec1.item(), n)
            validate_loss += loss.item()
            postfix = {'validate_loss': '%.6f' % (validate_loss / (i + 1)), 'validate_acc': '%.6f' % val_top1.avg}
            validate_loader.set_postfix(log=postfix)

            # ternsorboard 曲线绘制
            if os.path.exists(tensorboard_path) == False: 
                os.mkdir(tensorboard_path)    
            writer = SummaryWriter(tensorboard_path)
            writer.add_scalar('Validate/Loss', loss.item(), epoch)
            writer.add_scalar('Validate/Accuracy', val_top1.avg, epoch)
            writer.flush()
        val_acc = val_top1.avg
    return val_acc

In [35]:
validate(epoch, val_loader,device,model,criterion,"visual")

  0%|          | 0/785 [00:00<?, ?it/s]

89.37954164520389